In [1]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import config
from parse_functions import *
# ссылка на основной файл гугл докс (на каждом лите отдельный клиент со ссылками на файлы с данными)
base_link = config.base_link


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [3]:
# функция, которая на входе принимает 1 строку из основного датаФрейма
# строка содержит название отчета и ссылку на гугл докс с данными
# name - название клиента
# report - название отчета
def get_parse_reports(df, name, report):
    # т.к. в одном гугл доксе может быть много отчетов на разных листах, мы создаем словарь
    tmp_dict = {}
    # забираем ссылку на гугл докс, из нее получаем ИД и забираем данные 
    content = get_data_from_sheet(df['report_link'][0])
    # проверяем - если есть доступк к гугл доксу, то запускаем парсинг данных
    # но может прийти ошибка, если НЕТ общего доступа
    try:
        # забираем названия листов в гугл доксе
        target_sheet_names = pd.ExcelFile(content)
        # проходим через цикл по названиям всех листов
        for target_sheet in target_sheet_names.sheet_names:
            # если в названиии листа есть слово чек-лист, то вызываем функцию для парсинга данных
            if 'чек-лист' in target_sheet.lower():
                # final_df = parse_check_list_report(content, target_sheet, name)
                # для отчетов калиниград ключом является название отчета из основного файла
                if 'калининград' in name.lower():
                    tmp_dict[report] = parse_check_list_report(content, target_sheet, name, report)
                else:
                # для всех остальных ключ - это название листа
                    tmp_name = str(target_sheet).lower()
                    # print(tmp_name)
                    tmp_dict[tmp_name] = parse_check_list_report(content, target_sheet, name, report)
            # для отчетов СРМ забираем данные только с листа Показатели
            if 'срм' in report.lower() and target_sheet.lower()=='показатели':
                tmp_dict[report] = parse_crm_report(content, target_sheet, name, report)
                
            # для списка Сотрудников забираем данные только с листа Показатели    
            if 'сотрудники' in report.lower() and target_sheet.lower()=='компании':
                tmp_dict[report] = parse_employees_report(content, target_sheet, name, report)

            if 'цели' in report.lower():
                report_name = report + '_' + target_sheet.lower() 
                tmp_dict[report_name] = parse_targets_plan_report(content, target_sheet, name, report_name)
            
        return tmp_dict
    # если доступа нет, то возвращаем инфо -к какому отчету НЕ получилось подключиться
    except:
        print(f'Нет доступа к файлу: Клиента - {name} / Отчет: {report}')

In [4]:
# забираем данные из основной управленческой таблицы
content = get_data_from_sheet(base_link)
sheet_names = pd.ExcelFile(content)

In [5]:
sheet_names.sheet_names

['Калининград МореАвто(ЮтоКарс)', 'Екатеринбург АтлантикПРО', 'АвтоБелогорье']

In [6]:
# создаем словарь, в который сохраним результаты парсинга всех отчетов
tmp_reports_dict = {}
# проходим через цикл по всем листам в управленческой таблице
for name in sheet_names.sheet_names:
    # print(name)
    
    cols_range = 'A:B' # задаем диапазон полей, которые нам нужны
    df = pd.read_excel(content, sheet_name=name, usecols=cols_range, names=['report_name', 'report_link'])
    # нормализуем назания отчетов - приводим в нижний регистр, обрезаем по краям, избаляемся от технических символов(перенос строки и тд)
    df['report_name'] = df['report_name'].str.lower().str.strip().str.replace('\n', ' ')
    # создаем отдельный список с названиями отчетов и проодим по нему через цикл
    reports_list = list(df['report_name'])
    # запускаем блок парсинга отчетов
    for report in reports_list:
        if 'чек' in report.lower() or  'срм' in report.lower() or 'сотрудники' in report.lower() or 'цели' in report.lower():
            parse_info = df[df['report_name']==report].reset_index(drop=True)
            # создаем спец. ключ для словаря
            key = name + '_' + report
            tmp_reports_dict[key] = get_parse_reports(parse_info, name, report)
            print(key)
       


Калининград МореАвто(ЮтоКарс)_чек-лист калининград ап
Калининград МореАвто(ЮтоКарс)_чек-лист калининград оп
Калининград МореАвто(ЮтоКарс)_чек-лист калининград выкуп
Калининград МореАвто(ЮтоКарс)_чек-лист калининград парсинг
Калининград МореАвто(ЮтоКарс)_срм ап
Калининград МореАвто(ЮтоКарс)_срм оп
Калининград МореАвто(ЮтоКарс)_цели
Калининград МореАвто(ЮтоКарс)_сотрудники
чек-лист ап
чек-лист чаты-звонки
чек-лист комиссия
чек-лист чаты
Екатеринбург АтлантикПРО_чек-лист
Екатеринбург АтлантикПРО_срм чаты_звонки
Екатеринбург АтлантикПРО_цели
Екатеринбург АтлантикПРО_сотрудники
Екатеринбург АтлантикПРО_срм ап
чек-лист оп exeed белгород
чек-лист оп faw_jac белгород
чек-лист оп omoda_jaecoo старый
чек-лист оп jetour старый оскол
чек-лист оп лада белгород
чек-лист ап
АвтоБелогорье_чек-лист автобелогорье
АвтоБелогорье_цели
АвтоБелогорье_сотрудники
АвтоБелогорье_срм лада
АвтоБелогорье_срм джетур
АвтоБелогорье_срм омода
Нет доступа к файлу: Клиента - АвтоБелогорье / Отчет: срм фав
АвтоБелогорье_с

In [7]:
for key in tmp_reports_dict.keys():
    print(key)

Калининград МореАвто(ЮтоКарс)_чек-лист калининград ап
Калининград МореАвто(ЮтоКарс)_чек-лист калининград оп
Калининград МореАвто(ЮтоКарс)_чек-лист калининград выкуп
Калининград МореАвто(ЮтоКарс)_чек-лист калининград парсинг
Калининград МореАвто(ЮтоКарс)_срм ап
Калининград МореАвто(ЮтоКарс)_срм оп
Калининград МореАвто(ЮтоКарс)_цели
Калининград МореАвто(ЮтоКарс)_сотрудники
Екатеринбург АтлантикПРО_чек-лист
Екатеринбург АтлантикПРО_срм чаты_звонки
Екатеринбург АтлантикПРО_цели
Екатеринбург АтлантикПРО_сотрудники
Екатеринбург АтлантикПРО_срм ап
АвтоБелогорье_чек-лист автобелогорье
АвтоБелогорье_цели
АвтоБелогорье_сотрудники
АвтоБелогорье_срм лада
АвтоБелогорье_срм джетур
АвтоБелогорье_срм омода
АвтоБелогорье_срм фав
АвтоБелогорье_срм эксид


In [8]:
for key in tmp_reports_dict['Екатеринбург АтлантикПРО_срм ап'].keys():
    print(key)

срм ап


In [9]:
tmp_reports_dict['Екатеринбург АтлантикПРО_срм ап']['срм ап'].head()

,дата,команда,кол-во лидов,кол-во согласившихся на визит,кол-во состоявшихся визитов,client,dashboard
0,2024-07-01,"Репина, 13",,,,Екатеринбург АтлантикПРО,срм ап
1,2024-08-07,"Таганская, 77",6,,2,Екатеринбург АтлантикПРО,срм ап
2,2024-08-07,"Репина, 13",,,,Екатеринбург АтлантикПРО,срм ап
3,2024-08-08,"Таганская, 77",12,,3,Екатеринбург АтлантикПРО,срм ап
4,2024-08-08,"Репина, 13",,,,Екатеринбург АтлантикПРО,срм ап


In [10]:
tmp_reports_dict['Калининград МореАвто(ЮтоКарс)_чек-лист калининград ап']['чек-лист калининград ап'].head(3)

,номер события,ссылка на звонок,источник контакта,дата звонка,фио менеджера,менеджер представился по имени,менеджер назвал компанию,менеджер уточнил сроки покупки,менеджер предложил traid-in( с выгодой),менеджер предложил кредит(с выгодой),"менеджер создал срочность (много интересуются, акция)",предложение другого авто (если не назначена встреча),формат ответа на вопрос о цене,менеджер спросил мнение клиента о предложении,фиксация возражения клиента,менеджер аргументирует возражение клиента,менеджер предложил визит в салон (с объяснением для чего),менеджер договорился о следующем шаге с клиентом,оценка звонка,клиент согласился на визит,по сделке стоит актуальная задача,"причины, почему не согласился на визит",марка авто,комментарии,статус,сценарий соблюден,команда,client,dashboard
3,,,,,Вес вопроса,0.01,0.01,0.05,0.05,0.05,0.08,0.06,0.16,0.12,да (какое?)/нет,0.15,0.13,0.13,1,да/нет,да/нет,,,,,,,Калининград МореАвто(ЮтоКарс),чек-лист калининград ап
0,ЮТ-0000648,https://disk.yandex.ru/d/xPmm0z3wfnk5LQ,Авито,2024-01-03,Маркарян Эдгар Андраникович,0.01,0,0.05,0,0.05,0,0.06,0.16,0,нет,0.15,0.13,0.13,0.74,да,да,,Джили,,Одобрен,,Тойота,Калининград МореАвто(ЮтоКарс),чек-лист калининград ап
1,ЮТ-0000667,https://disk.yandex.ru/d/yNlQ66RxM3W04Q,Авито,2024-01-03,Сухоруков Алексей Викторович,0.01,0,0,0.05,0.05,0,0.06,0.16,0,нет,0.15,0.13,0.13,0.74,да,,,Фольксваген пассат или Мерседес,,Одобрен,,Тойота,Калининград МореАвто(ЮтоКарс),чек-лист калининград ап


In [11]:
tmp_reports_dict['АвтоБелогорье_сотрудники']['сотрудники'].head(3)

,флаг,фио,логин,почта,пароль,exeed,faw_jac,omoda_jaecoo,лада,jetour,общий дашборд,client,dashboard
0,,Евгений Образцов,exeed,Sales-director@belog­orie-exeed.ru,UYJCPItCeV6mbv%3Ssdoak3f,https://дашборд.online/exeed-2,,,,,,АвтоБелогорье,сотрудники
1,,Мельничук Роман,r.melnichyk,r.melnichyk@avto-belogorie.ru,L)G6T3bGf1bChHjME@8QTtj(,,https://дашборд.online/faw_jac-2,,,,,АвтоБелогорье,сотрудники
2,,Плахова Ольга,lada,sales-director@lada-sputnik.ru,84Hz)HCq!8VNdQW%0&$#$Yc(,,,,https://дашборд.online/lada-2,,,АвтоБелогорье,сотрудники


In [ ]:
for key in tmp_reports_dict['Калининград МореАвто(ЮтоКарс)_цели'].keys():
    print(key)

In [12]:
tmp_reports_dict['Калининград МореАвто(ЮтоКарс)_цели']['цели_ап']

,дата начала,дата окончания,название,план,client,dashboard
0,2024-09-12,2024-09-18,,,Калининград МореАвто(ЮтоКарс),цели_ап
1,2024-09-12,2024-09-18,,,Калининград МореАвто(ЮтоКарс),цели_ап
2,2024-09-12,2024-09-18,,,Калининград МореАвто(ЮтоКарс),цели_ап
3,2024-09-19,2024-09-25,"Менеджер создал срочность (много интересуются, акция)",0.5,Калининград МореАвто(ЮтоКарс),цели_ап
4,2024-09-19,2024-09-25,Менеджер аргументирует возражение клиента,0.65,Калининград МореАвто(ЮтоКарс),цели_ап
5,2024-09-19,2024-09-25,Менеджер предложил визит в салон (с объяснением для чего),0.6,Калининград МореАвто(ЮтоКарс),цели_ап
6,2024-09-26,2024-10-02,"Менеджер создал срочность (много интересуются, акция)",0.5,Калининград МореАвто(ЮтоКарс),цели_ап
7,2024-09-26,2024-10-02,Менеджер аргументирует возражение клиента,0.65,Калининград МореАвто(ЮтоКарс),цели_ап
8,2024-09-26,2024-10-02,Менеджер предложил визит в салон (с объяснением для чего),0.6,Калининград МореАвто(ЮтоКарс),цели_ап
9,2024-10-03,2024-10-09,,,Калининград МореАвто(ЮтоКарс),цели_ап
